In [ ]:
alephhome = '/home/hansu/aleph/aleph-node'

In [ ]:
import sys
import shutil
from os.path import join

sys.path.append(join(alephhome, 'local-tests'))
from chainrunner import Chain, Seq, generate_keys, check_finalized, check_version

In [ ]:
desk = '/home/hansu/aleph/desk'
workdir = join(desk, 'workdir')

testbin = join(desk, 'bin', 'aleph-node-5.2')
binary = join(desk, 'bin', 'aleph-node-6.0')

runtime = join(desk, 'bin', 'aleph_runtime.compact.wasm.20')
forkoff = join(alephhome, 'fork-off/target/release/fork-off')
cliain = join(alephhome, 'bin/cliain/target/release/cliain')

testnet_ws = 'wss://ws.test.azero.dev'

In [ ]:
PORT = 30334
WS_PORT = 9944
RPC_PORT = 9933

In [ ]:
phrases = ['//Alice', '//Bob', '//Charlie', '//Dave']
keys = generate_keys(binary, phrases)

In [ ]:
#shutil.rmtree(workdir, True)
chain = Chain(workdir)
chain.bootstrap(testbin, keys.values(),
                chain_type='live',
                sudo_account_id=keys[phrases[0]],
                raw=True)

In [ ]:
chain.fork(forkoff, testnet_ws)

In [ ]:
chain.set_flags('validator',
                'unsafe-ws-external',
                'unsafe-rpc-external',
                'no-mdns',
                port=Seq(PORT),
                ws_port=Seq(WS_PORT),
                rpc_port=Seq(RPC_PORT),
                unit_creation_delay=500,
                execution='Native',
                rpc_cors='all',
                rpc_methods='Unsafe',
               )

In [ ]:
addresses = [n.address() for n in chain]
chain.set_flags(bootnodes=addresses[0], public_addr=addresses)

In [ ]:
chain.start('node')

In [ ]:
check_finalized(chain)

In [ ]:
check_version(chain)

In [ ]:
def restart(i):
    chain[i].stop()
    chain.set_binary(binary, nodes=[i])
    chain[i].start(f'new{i}')

In [ ]:
restart(2)

In [ ]:
restart(1)

In [ ]:
restart(3)

In [ ]:
restart(0)

In [ ]:
check_finalized(chain)

In [ ]:
check_version(chain)

In [ ]:
chain.update_runtime(cliain, phrases[0], runtime)

In [ ]:
check_version(chain)

In [ ]:
check_finalized(chain)

In [ ]:
chain.stop()